## 4 - Convolutional Sentiment Analysis

> Ở notebooks trước, chúng ta đã đạt được accuracy xấp xỉ 85% bằng cách sử dụng RNNs và áp dụng mô hình trong paper [Bag of Tricks for Efficient Text Classification](https://arxiv.org/pdf/1607.01759.pdf). Trong notebook này, chúng ta sẽ cùng nhau tìm hiểu và áp dụng *convolutional neural network (CNN) để xây dựng một mô hình phân tích cảm xúc, áp dụng mô hình trong bài báo [Convolutional Neural Networks for Sentence Classification](https://arxiv.org/pdf/1408.5882.pdf)*.

> LƯU Ý: Notebook này chỉ tóm gọn về convolutional neural network. Để biết thêm chi tiết về CNNs, bạn có thể tham khảo [CS231n: Convolutional Neural Networks for Visual Recognition](http://cs231n.github.io/convolutional-networks/). và [Convolutional Neural Networks Coursera](https://www.coursera.org/learn/convolutional-neural-networks), hoặc [Convolutional Neural Networks blog](https://ujjwalkarn.me/2016/08/11/intuitive-explanation-convnets/).

> Thông thường, CNNs được sử dụng trong phân tích ảnh. CNNs thường được tạo bởi một hoặc nhiều lớp tích chập - *convolutional* layers, theo sau là một hoặc nhiều *linear layers*. *Convolutional* layers sử dụng các *filters* (còn được gọi là *kernels* hoặc *receptive fields*) quét ngang qua một bức ảnh và sinh ra một bức ảnh mới đã được xử lý. Bức ảnh mới này có thể đưa qua các lớp covolutional hoặc linear tùy bài toán. Mỗi filter đều có kích thước, ví dụ một filter 3x3 có chiều dài là 3, rộng 3 và tổng cộng có 9 trọng số. Trong xử lý ảnh truyền thống, các trọng số này sẽ được tính toán bằng tay. Tuy nhiên, với CNNs, các trọng số này được học thông qua *backpropagation*.

> Ý tưởng của việc học các trọng số là: mỗi *convolutional* layer hoạt động như một bộ trích xuất đặc trưng - *feature extractors*, trích xuất những phần quan trọng trong bức ảnh. Ví dụ, nếu sử dụng CNN để phát hiện khuôn mặt, CNN sẽ nhìn vào những đặc trưng quan trọng của khuôn mặt như: mắt, mũi, miệng, lông mày, ....

> Thế, tại sao lại sử dụng CNNs cho văn bản? Tương tự như việc xử lý ảnh với filter 3x3, một filter 1x2 có thể nhanh chóng kiểm tra 2 từ liền nhau trong một đoạn văn bản, ví dụ: bi-grams. Ở Notebook trước, chúng ta sử dụng FastText model - sử dụng bi-grams bằng cách chèn chúng vào cuối mỗi câu. Lần này, chúng ta sẽ sử dụng CNNs để học bi-grams (1x2 filter) một cách tự động, bên cạnh đó còn học tri-grams (1x3 filter), ..., n-grams (1xn filter) trong văn bản. 

> Trực giác cho thấy rằng: nếu có một số bi-grams, tri-grams và n-grams nhất định xuất hiện trong bài đánh giá, đó sẽ là dấu hiệu tốt để đưa ra kết luận cuối cùng cho bài đánh giá là tích cực (pos) hay tiêu cực (neg)

### Preparing Data

> Ta xây dựng tập dữ liệu tương tự notebook 2. <br>
> Do convolutional layers yêu cầu số chiều của batch phải nằm trước nên ta sử dụng `batch_first=True` khi khởi tạo `DataLoader`.

In [1]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import random
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np


SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                    tokenizer_language='en_core_web_sm',
                    batch_first = True)

LABEL = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

> Xây dựng vocab và load pre-trained word embeddings.

In [2]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data,
                    max_size = MAX_VOCAB_SIZE,
                    vectors = "glove.6B.100d",
                    unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

> Tương tự như các notebook trước, ta tạo iterators.

In [3]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = \
    data.BucketIterator.splits(
        (train_data, valid_data, test_data),
        batch_size = BATCH_SIZE,
        device = device) 

### Build the Model

> Đầu tiên, chúng ta cần phải hình dùng cách áp dụng CNNs cho text. Trong trường hợp ảnh xám, mỗi bức ảnh sẽ có 2 chiều (ảnh màu sẽ có nhiều dimensional hơn ảnh xám) trong khi đó, text có 1 dimensional. Tuy nhiên, ta đều biết rằng bước đầu tiên trong các NLP pipelines là biến đổi các từ thành word embeddings. Đây là cách ta biểu diễn các words ở dạng 2 dimensions, các từ nằm theo 1 trục, trục còn lại là các phần tử của word embeddings. Vì vậy, ta có thể coi word embeddings như là một bức ảnh xám. Quan sát word embeddings biểu diễn 1 câu dưới đây: <br>
>> I hate this film. <br>

![LSTM](./images/4_example_word_embeddings.PNG "LSTM")



> Ta có thể sử dụng filter có kích thước **[n x emb_dim]**. Filter này sẽ bao quát hoàn toàn n chuỗi từ, do chiều rộng của filter là `emb_dim`. Trong bức ảnh bên dưới, word vectors được biểu diễn bởi màu xanh, ở đây ta có 4 từ, mỗi từ có 5 dimensional embeddings, tạo nên 1 tensor 2 chiều có kích thước **[4x5]**. Giả sử ta muốn tạo 1 filter sẽ học 2 từ 1 lúc (bi-grams) nên kích thước của nó sẽ là **[2x5]** (filter có màu vàng) và mỗi phần tử trong filter có một *weight* tương ứng (trong trường hợp này là 10 *weights* do filter có size 2x5). Đầu ra của 1 filter sẽ là 1 số thực, được tính bằng cách lấy tổng có trọng số của tất cả các phần tử được quét bởi filter.

![LSTM](./images/4_filter_figure1.PNG "LSTM")


> Sau đó, filter di chuyển xuống dưới dọc theo câu để kiểm tra bi-gram tiếp theo và sinh ra một đầu ra mới.

![LSTM](./images/4_filter_figure2.PNG "LSTM")


> Cuối cùng, filter di chuyển xuống cuối và ta thu được đầu ra của bi-gram cuối cùng.

![LSTM](./images/4_filter_figure3.PNG "LSTM")


> Trong trường hợp này (và các trường hợp mà chiều rộng của filter bằng với chiều rộng của "ảnh" - tức kích thước của 1 embedding vector), đầu ra sẽ là một vector có số phần tử bằng "chiều cao của ảnh" - trong trường hợp này là chiều dài của câu trừ đi chiều cao của filter và cộng thêm một:
>> kích thước đầu ra = chiều dài của câu - chiều cao của filter + 1
>>> trong trường hợp này sẽ là: 4 - 2 + 1 = 3

> Các model có CNNs đều có rất nhiều filter. Ý tưởng ở đây là **mỗi filter sẽ học một thuộc tính khác nhau để trích xuất**. Ở ví dụ trên, với filter có kích thước **[2 x emb_dim]** sẽ học được sẽ xuất hiện của mỗi cặp từ (hay còn gọi là bi-grams).

> Model chúng ta sẽ xây dựng còn sử dụng các filter có kích thước khác nhau, chiều cao lần lượt là 3, 4 và 5, mỗi kích thước có 100 filters. Trực giác cho thấy, sự xuất hiện của các tri-grams, 4-grams và 5-grams nhất định sẽ ảnh hưởng đến kết quả dự đoán cuối cùng.

> Bước tiếp theo, ta sẽ sử dụng *pooling* (ở đây sẽ là *max pooling*) ở đầu ra của *convolutional layers*. Hình dưới là hình minh họa của *max pooling*:

![LSTM](./images/4_maxpooling_figure4.PNG "LSTM")


> Ý tưởng của *max pooling* là giá trị lớn nhất *có vai trò quan trọng hơn* trong việc xác định cảm xúc của bài đánh giá, tương ứng với **n-grams quan trọng hơn trong bài đánh giá**. Nhưng mà làm sao để biết n-grams nào quan trọng hơn? May mắn là với backpropagation, **weights** của filters sẽ thay đổi khi gặp các n-grams nhất định, biểu diễn cảm xúc của bài đánh giá rõ rệt hơn, do đó giá trị đầu ra của filter sẽ cao hơn. Giá trị cao này sẽ đi qua maxpooling nếu nó là giá trị cao nhất.

> Do model của chúng ta có 100 filters với 3 kích thước khác nhau, có nghĩa chúng ta có 300 n-grams khác nhau mà model *nghĩ* nó là quan trọng. Chúng ta ghép **chúng** lại với nhau tạo thành *một single vector* và truyền vào *linear layers* để dự đoán. Ta có thể nghĩ *linear layer* này được "tăng cường độ uy tín" từ mỗi 300 n-grams và đưa ra kết quả cuối cùng.

##### Implementation Details

> Chúng ta sẽ triển khai *convolutional layers* bằng `nn.Conv2d`. Đối số `in_channels` là số lượng kênh đầu vào, với ảnh màu (RGB), số kênh sẽ là 3, tuy nhiên ta sử dụng word embeddings nên chỉ có 1 kênh. Đối số `out_channels` là số lượng filters, còn `kernel_size` là kích thước của filters. Mỗi `kernel_size` có kích thước là **[n x emb_dim]** với n là số của n-grams.

> Trong Pytorch, RNNs yêu cầu đầu vào có batch demension ở ví trí thứ 2, còn CNNs yêu cầu batch dimension ở vị trí đầu tiên - ta không cần hoán vị data do ta đã set `batch_first = True` trong `TEXT`. Sau đó, chúng ta sẽ truyền các sentence vào một embedding layer để lấy embeddings. Đối số thứ 2 trong `nn.Conv2d` phải là channel dimension. Tuy nhiên embeddings chỉ có 2 chiều, chưa có channel dimension nên ta dùng `unsqueeze` để tạo thêm channel dimension. Điều này khớp với tham số `in_channels = 1` khi khởi tạo *convolutional layers*.

> Tiếp đến, ta truyền tensors qua convolutional và pooling layers, sử dụng `ReLU` activation function sau convolutional layers. pooling layers có 1 đặc tính khá nice đó là nó có thể tự động xử lý các câu có chiều dài khác nhau. Kích thước đầu ra của convolutional layers phụ thuộc vào kích thước đầu vào, và batches khác nhau sẽ chứa các câu có độ dài khác nhau. Nếu không có maxpooling layer, đầu vào của linear layer sẽ phụ thuộc vào kích thước của câu đầu vào (và đương nhiên ta không muốn điều đó xảy ra). Có một phương pháp khác để xử lý vấn đề này là ta sẽ đệm các câu sao cho chúng có cùng độ dài. Tuy nhiên với maxpooling, ta luôn biết trước được kích thước đầu vào của linear layer và nó bằng số lượng filters.

 >**NOTE**: Có một ngoại lệ, xảy ra khi câu của ta ngắn hơn filters có kích thước lớn nhất. Trong trường hợp này, chúng ta cần đệm câu sao cho chúng có độ dài bằng với filters có size lớn nhất.

> Cuối cùng ta thực hiện *dropout* ở vị trí đầu ra của filters đã được ghép nối với nhau và truyền qua linear layer để dự đoán.

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):

    def __init__(self, vocab_size, embedding_dim, n_filters, filters_size, output_dim, dropout, pad_idx):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

        self.conv_0 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filters_size[0], embedding_dim))

        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filters_size[1], embedding_dim))


        self.conv_2 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filters_size[2], embedding_dim))


        self.fc = nn.Linear(len(filters_size) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        # text = [batch size, sent len]
        embedded = self.embedding(text)

        # embedded = [batch size, sent len, emb dim]

        embedded = embedded.unsqueeze(1)

        # embedded = [batch size, 1, sent len, emb dim]

        conved0 = F.relu(self.conv_0(embedded))
        conved1 = F.relu(self.conv_1(embedded))
        conved2 = F.relu(self.conv_2(embedded))

        # Do sử dụng CNNs và filter có kích thước [n x emb_dim] nên sau khi qua conv ta có chiều như sau:
        # convedn = [batch size, n_filters, sent len - filter_sizes[n] + 1, 1]

        conved0 = conved0.squeeze(3)
        conved1 = conved1.squeeze(3)
        conved2 = conved2.squeeze(3)

        # convedn = [batch size, n_filters, sent len - filter_sizes[n] + 1]

        pooled0 = F.max_pool1d(conved0, conved0.shape[2]).squeeze(2)
        pooled1 = F.max_pool1d(conved1, conved1.shape[2]).squeeze(2)
        pooled2 = F.max_pool1d(conved2, conved2.shape[2]).squeeze(2)

        # pooledn = [batch_size, n_filters]

        cat = self.dropout(torch.cat((pooled0, pooled1, pooled2), dim = 1))

        # cat = [batch_size, n_filters * len(filter_sizes)]

        final_output = self.fc(cat)

        # final_output = [batch_size, output_dim]

        return final_output

> Class `CNN` hiện tại chỉ có 3 lớp convolutional, tuy nhiên ta có thể cải thiện để model tổng quát hơn và nhận số lượng lớp convolutional bất kỳ.

> Để thực hiện được điều đó, ta đưa các convolutional vào một `nn.ModuleList`, đây là một hàm được dùng để lưu trữ danh sách các `nn.Module`s trong Pytorch. Nếu ta chỉ dùng `list` trong Python, các modules bên trong list sẽ không thể được nhìn thấy bởi bất kỳ module nào bên ngoài list đó => gây ra lỗi.

In [5]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

> Thực tế, ta có thể sử dụng 1 - dimensional convolutional layers, khi đó embedding dimension sẽ là chiều sâu của filter và số lượng tokens trong câu sẽ là chiều rộng.

In [6]:
class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.permute(0, 2, 1)
        
        #embedded = [batch size, emb dim, sent len]
        
        conved = [F.relu(conv(embedded)) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        
        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [7]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,620,801 trainable parameters


> Load pre-trained embeddings

In [9]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1386,  0.1180,  0.3534,  ...,  0.1226,  0.5973, -0.1702],
        [-0.0786,  0.0541, -0.0993,  ...,  0.2565, -0.1874, -0.4428],
        [-0.3617,  0.6201,  0.1105,  ...,  0.2994, -0.5920,  1.0949]])

> Khởi tạo weights bằng 0 cho các tokens chưa biết và các padding tokens.

In [10]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## Train the Model

In [11]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [12]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [13]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

> Nhớ là dùng `model.eval()` khi test để tắt dropout.

In [14]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [15]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [16]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './models/tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

c:\Users\longln3\Anaconda3\envs\python3.7.6\lib\site-packages\torch\nn\functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 01 | Epoch Time: 6m 43s
	Train Loss: 0.651 | Train Acc: 61.70%
	 Val. Loss: 0.521 |  Val. Acc: 76.07%
Epoch: 02 | Epoch Time: 7m 32s
	Train Loss: 0.433 | Train Acc: 80.22%
	 Val. Loss: 0.366 |  Val. Acc: 83.92%
Epoch: 03 | Epoch Time: 6m 48s
	Train Loss: 0.306 | Train Acc: 87.42%
	 Val. Loss: 0.328 |  Val. Acc: 85.47%
Epoch: 04 | Epoch Time: 6m 7s
	Train Loss: 0.220 | Train Acc: 91.33%
	 Val. Loss: 0.325 |  Val. Acc: 86.08%
Epoch: 05 | Epoch Time: 6m 45s
	Train Loss: 0.155 | Train Acc: 94.28%
	 Val. Loss: 0.328 |  Val. Acc: 86.33%


In [17]:
model.load_state_dict(torch.load('./models/tut4-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.346 | Test Acc: 85.33%


## User Input

> **NOTE**: Như đã đề cập từ trước, chiều dài của input sentence tối thiểu phải bằng với chiều cao của filters được sử dụng. Do đó ta điều chỉnh hàm `predict_sentiment` để có thể nhận đầu vào có kích thước nhỏ. Nếu *tokenized input sentence* có độ dài nhỏ hơn `min_len`, ta sẽ đệm thêm để nó có chiều dài bằng `min_len`.

In [18]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence, min_len = 5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += [''] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [19]:
predict_sentiment(model, "This film is terrible")

0.09431809186935425

In [20]:
predict_sentiment(model, "This film is great")

0.9623662829399109